### Per poter scaricare il dataset dobbiamo partire con questo comando
## !pip install kaggle --upgrade

In [ ]:
# Facciamo il login tramite token su Kaggle

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
# scarichiamo il dataset

!kaggle competitions download -c dogs-vs-cats

In [2]:
# Import

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [ ]:
print(tf.__version__)

In [32]:
# Spacchettiamo  il dataset dallo zip

main = "dogs-vs-cats.zip"
train = "train.zip"
test = "test1.zip"
cartella_principale = "DOGvsCATS/"
files = "dataset/"

import zipfile

if os.path.isfile(train) and os.path.isfile(test) == True:
    pass
else:
    with zipfile.ZipFile(main,"r") as zipp:
        zipp.extractall()
    if os.path.isdir("train") and os.path.isdir("test1") == True:
        pass
    else:
        with zipfile.ZipFile(train, "r") as zipp:
            zipp.extractall(files)

        with zipfile.ZipFile(test, "r") as zipp:
            zipp.extractall(files)

In [3]:
# Creiamo un dataframe per poter modellare i dati

dataset_train_path = "dataset/train/"

nomefile = os.listdir(dataset_train_path)
specie = [x.split(".")[0] for x in nomefile]

dati = pd.DataFrame({"Nome_File": nomefile, "Specie":specie})

#dati["Specie"] = dati["Specie"].replace(["cat","dog"],[0,1])


dati

,Nome_File,Specie
0,cat.0.jpg,cat
1,cat.1.jpg,cat
2,cat.10.jpg,cat
3,cat.100.jpg,cat
4,cat.1000.jpg,cat
...,...,...
24995,dog.9995.jpg,dog
24996,dog.9996.jpg,dog
24997,dog.9997.jpg,dog
24998,dog.9998.jpg,dog


In [6]:
# main = "dogs-vs-cats.zip"
# train = "train.zip"
# test = "test1.zip"
# cartella_principale = "DOGvsCATS/"
# files = "working/"

# train_path = "working/train"
# test_path = "working/test"

train_path = "working/train/"
test_path = "working/test/"

# Separiamo le immagini per cani e gatti
from os import makedirs, listdir
from random import random
from shutil import copyfile

cartel= [train_path, test_path]
# Creazione sottocartelle
for i in cartel:
    nome_cartel=["/cani/","/gatti/"]
    for j in nome_cartel:
        nuova_cartel = i+j
        makedirs(nuova_cartel, exist_ok=True)
        
# Percentuale di immagini per validare il training
valutazione = 0.2

# Copia delle immagini nelle sottocartelle
src_dir = "dataset/"
source = "dataset/train/"
for immagine in listdir(source):
    src=src_dir+"train/"+immagine
    destinazione="working/train/"
    if random()< valutazione: 
        destinazione="working/test/"
    if immagine.startswith("cat"):
        dest = destinazione + "gatti/" + immagine
        copyfile(src, dest)
    elif immagine.startswith("dog"):
        dest = destinazione + "cani/" + immagine
        copyfile(src, dest)


path1 = "working/train/gatti"
path2= "working/train/cani"
path3= "working/test/gatti"
path4= "working/test/cani"

print('Ci sono ' ,len(os.listdir(path1)),' immagini di gatti in training')
print('Ci sono ' ,len(os.listdir(path2)),' immagini di cani in training')
print('Ci sono ' ,len(os.listdir(path3)),' immagini di gatti in test')
print('Ci sono ' ,len(os.listdir(path4)),' immagini di cani in test')

Then number of cat images in training data is 11979
Then number of dog images in training data is 12012
Then number of cat images in validation data is 4584
Then number of dog images in validation data is 4569


In [22]:
# creiamo i generatori
grandezza_immagine = 128


train_gen = keras.utils.image_dataset_from_directory(
    directory= "working/train",
    labels="inferred",
    label_mode="binary",
    batch_size=32,
    image_size=(grandezza_immagine,grandezza_immagine)
)

test_gen = keras.utils.image_dataset_from_directory(
    directory= "working/test",
    labels="inferred",
    label_mode="binary",
    batch_size=32,
    image_size=(grandezza_immagine,grandezza_immagine)
)                                                                  

Found 23991 files belonging to 2 classes.
Found 9153 files belonging to 2 classes.


In [24]:
# Creiamo il modello sequenziale
modello = keras.Sequential()
modello.add(layers.Conv2D(32, (3,3), input_shape=(grandezza_immagine,grandezza_immagine,3), activation="relu"))
modello.add(layers.MaxPooling2D((2,2)))
modello.add(layers.Conv2D(64,(3,3),activation="relu"))
modello.add(layers.MaxPooling2D((2,2)))
modello.add(layers.Conv2D(128,(3,3),activation="relu"))
modello.add(layers.MaxPooling2D((2,2)))
modello.add(layers.Conv2D(128,(3,3),activation="relu"))
modello.add(layers.Flatten())
modello.add(layers.Dense(512, activation="relu"))
modello.add(layers.Dense(1, activation="softmax"))

# Compiliamo il modello
modello.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

modello.summary()

In [25]:
# Addestriamo il modello
modello.fit(train_gen, epochs = 10, validation_data=test_gen)

Epoch 1/10


750/750 [==============================] - 304s 404ms/step - loss: 1.9196 - accuracy: 0.4993 - val_loss: 0.6823 - val_accuracy: 0.5008
Epoch 2/10
750/750 [==============================] - 298s 397ms/step - loss: 0.6890 - accuracy: 0.4993 - val_loss: 0.7045 - val_accuracy: 0.5008
Epoch 3/10
750/750 [==============================] - 303s 404ms/step - loss: 0.7276 - accuracy: 0.4993 - val_loss: 0.6924 - val_accuracy: 0.5008
Epoch 4/10
750/750 [==============================] - 295s 393ms/step - loss: 0.6933 - accuracy: 0.4993 - val_loss: 0.6950 - val_accuracy: 0.5008
Epoch 5/10
750/750 [==============================] - 296s 395ms/step - loss: 0.6948 - accuracy: 0.4993 - val_loss: 0.6917 - val_accuracy: 0.5008
Epoch 6/10
750/750 [==============================] - 294s 392ms/step - loss: 0.6926 - accuracy: 0.4993 - val_loss: 0.8365 - val_accuracy: 0.5008
Epoch 7/10
750/750 [==============================] - 285s 380ms/step - loss: 0.6919 - accuracy: 0.4993 - val_loss: 0.6905 - val_accura